In [1]:
#importing library
import pandas as pd
import numpy as np
import os

# Path

In [2]:
path = r'/Users/mehreenbecker/4.3 IC Data Import and Descriptive Analysis/Original File'

# Importing Data

In [3]:
ords_prods_merge = pd.read_pickle(os.path.join(path, 'ords_prods_merge_extra_columns.pkl'))

In [4]:
df = ords_prods_merge[:1_000_000]

# Checking Data

In [5]:
ords_prods_merge.shape

(36210749, 37)

In [6]:
ords_prods_merge.head()

product_id_x              product_name_x  aisle_id_x  department_id_x  \
0             1  Chocolate Sandwich Cookies          61               19   
1             1  Chocolate Sandwich Cookies          61               19   
2             1  Chocolate Sandwich Cookies          61               19   
3             1  Chocolate Sandwich Cookies          61               19   
4             1  Chocolate Sandwich Cookies          61               19   

   prices_x  Unnamed: 0_x  order_id  user_id_x  order_number_x  \
0       5.8          1987   3139998        138              28   
1       5.8          1989   1977647        138              30   
2       5.8         11433    389851        709               2   
3       5.8         11433    389851        709               2   
4       5.8         11433    389851        709               2   

   orders_day_of_week_x  ...  days_since_prior_order_y  First_order_y  \
0                     6  ...                       3.0          False   
1                     6  ...                      20.0          False   
2                     0  ...                       6.0          False   
3                     0  ...                       6.0          False   
4                     0  ...                       6.0          False   

   add_to_cart_order_y  reordered_y  _merge_y        price_range  \
0                  5.0          0.0      both  Mid-range product   
1                  1.0          1.0      both  Mid-range product   
2                 20.0          0.0      both  Mid-range product   
3                 25.0          0.0      both  Low-range product   
4                 16.0          0.0      both  Mid-range product   

   price_range_loc_y     busiest_day  busiest_days  busiest_period_of_day  
0  Mid-range product  Regularly busy  Regular days            Most orders  
1  Mid-range product  Regularly busy  Regular days         Average orders  
2  Mid-range product     Busiest day  Busiest days         Average orders  
3  Low-range product     Busiest day  Busiest days         Average orders  
4  Mid-range product     Busiest day  Busiest days         Average orders  

[5 rows x 37 columns]

# Subset

In [7]:
subset_stats = df.groupby("department_id_x").agg({
    "order_number_x": ["mean", "min", "max"]
})

In [8]:
full_means = ords_prods_merge.groupby("department_id_x")["order_number_x"].mean()
full_means

department_id_x
1     15.365874
2     17.172255
3     17.118844
4     17.753837
5     15.260670
6     16.356831
7     17.184587
8     15.305664
9     15.815569
10    20.078842
11    16.108891
12    15.826853
13    16.506275
14    16.754787
15    16.072112
16    17.598865
17    15.655170
18    19.202249
19    17.146566
20    16.432312
21    22.759759
Name: order_number_x, dtype: float64

In [10]:
ords_prods_merge["max_order"] = (
    ords_prods_merge.groupby("user_id_x")["order_number_x"].transform("max")
)

# Loyalty

In [11]:
ords_prods_merge.loc[ords_prods_merge["max_order"] > 40, "loyalty_flag"] = "Loyal Customer"
ords_prods_merge.loc[
    (ords_prods_merge["max_order"] <= 40) & (ords_prods_merge["max_order"] > 10), "loyalty_flag"
] = "Regular customer"
ords_prods_merge.loc[ords_prods_merge["max_order"] <= 10, "loyalty_flag"] = "New customer"

# Checking

In [13]:
ords_prods_merge[["user_id_x", "order_number_x", "max_order", "loyalty_flag"]].head(25)

user_id_x  order_number_x  max_order      loyalty_flag
0         138              28         32  Regular customer
1         138              30         32  Regular customer
2         709               2          5      New customer
3         709               2          5      New customer
4         709               2          5      New customer
5         709               2          5      New customer
6         764               1          3      New customer
7         764               3          3      New customer
8         777              16         26  Regular customer
9         825               3          9      New customer
10        910              12         12  Regular customer
11       1052              10         20  Regular customer
12       1052              15         20  Regular customer
13       1379               1         10      New customer
14       1480              36         43    Loyal Customer
15       1480              38         43    Loyal Customer
16       1480              38         43    Loyal Customer
17       1480              39         43    Loyal Customer
18       1494               6         13  Regular customer
19       1494               6         13  Regular customer
20       1494               6         13  Regular customer
21       1494               7         13  Regular customer
22       1494               7         13  Regular customer
23       1494               9         13  Regular customer
24       1540               4         28  Regular customer

In [15]:
ords_prods_merge.groupby("loyalty_flag")["prices_x"].mean()

loyalty_flag
Loyal Customer      10.494493
New customer        13.507039
Regular customer    12.821066
Name: prices_x, dtype: float64

# Average Spend User

In [16]:
ords_prods_merge["avg_spend_user"] = (
    ords_prods_merge.groupby("user_id_x")["prices_x"].transform("mean")
)

ords_prods_merge.loc[ords_prods_merge["avg_spend_user"] < 10, "spending_flag"] = "Low spender"
ords_prods_merge.loc[ords_prods_merge["avg_spend_user"] >= 10, "spending_flag"] = "High spender"

In [18]:
spend_freq = ords_prods_merge["spending_flag"].value_counts(dropna=False)
spend_freq, ords_prods_merge[["user_id_x", "avg_spend_user", "spending_flag"]].head(25)

(spending_flag
 Low spender     35449364
 High spender      761385
 Name: count, dtype: int64,
     user_id_x  avg_spend_user spending_flag
 0         138        6.935811   Low spender
 1         138        6.935811   Low spender
 2         709        7.913859   Low spender
 3         709        7.913859   Low spender
 4         709        7.913859   Low spender
 5         709        7.913859   Low spender
 6         764        4.972414   Low spender
 7         764        4.972414   Low spender
 8         777        6.935398   Low spender
 9         825        5.957576   Low spender
 10        910        6.680000   Low spender
 11       1052        7.162500   Low spender
 12       1052        7.162500   Low spender
 13       1379        5.813636   Low spender
 14       1480        5.098039   Low spender
 15       1480        5.098039   Low spender
 16       1480        5.098039   Low spender
 17       1480        5.098039   Low spender
 18       1494        6.886909   Low spender
 19  

# Order Frequency vs Days Since Prior Order

In [19]:
ords_prods_merge["ord_regularity_median"] = (
    ords_prods_merge.groupby("user_id_x")["days_since_prior_order_y"].transform("median")
)

In [20]:
ords_prods_merge.loc[
    ords_prods_merge["ord_regularity_median"] > 20, "frequency_flag"
] = "Non-frequent customer"
ords_prods_merge.loc[
    (ords_prods_merge["ord_regularity_median"] > 10) & (ords_prods_merge["ord_regularity_median"] <= 20),
    "frequency_flag",
] = "Regular customer"
ords_prods_merge.loc[
    ords_prods_merge["ord_regularity_median"] <= 10, "frequency_flag"
] = "Frequent customer"

# Checking Data

In [21]:
ords_prods_merge[
    ["user_id_x", "days_since_prior_order_y", "ord_regularity_median", "frequency_flag"]
].head(25)

user_id_x  days_since_prior_order_y  ord_regularity_median  \
0         138                       3.0                    6.5   
1         138                      20.0                    6.5   
2         709                       6.0                    8.0   
3         709                       6.0                    8.0   
4         709                       6.0                    8.0   
5         709                       6.0                    8.0   
6         764                       NaN                    9.0   
7         764                       9.0                    9.0   
8         777                      26.0                   26.0   
9         825                      30.0                   30.0   
10        910                      30.0                   30.0   
11       1052                      19.0                    9.0   
12       1052                      15.0                    9.0   
13       1379                       NaN                   13.0   
14       1480                       7.0                    7.0   
15       1480                       7.0                    7.0   
16       1480                       7.0                    7.0   
17       1480                      12.0                    7.0   
18       1494                       2.0                    6.0   
19       1494                       2.0                    6.0   
20       1494                       2.0                    6.0   
21       1494                       6.0                    6.0   
22       1494                       6.0                    6.0   
23       1494                      10.0                    6.0   
24       1540                       7.0                    7.0   

           frequency_flag  
0       Frequent customer  
1       Frequent customer  
2       Frequent customer  
3       Frequent customer  
4       Frequent customer  
5       Frequent customer  
6       Frequent customer  
7       Frequent customer  
8   Non-frequent customer  
9   Non-frequent customer  
10  Non-frequent customer  
11      Frequent customer  
12      Frequent customer  
13       Regular customer  
14      Frequent customer  
15      Frequent customer  
16      Frequent customer  
17      Frequent customer  
18      Frequent customer  
19      Frequent customer  
20      Frequent customer  
21      Frequent customer  
22      Frequent customer  
23      Frequent customer  
24      Frequent customer

In [22]:
freq_counts = ords_prods_merge["frequency_flag"].value_counts(dropna=False)
freq_counts

frequency_flag
Frequent customer        21722212
Regular customer          6833413
Non-frequent customer     4283747
NaN                       3371377
Name: count, dtype: int64

# Exporting

In [23]:
export_path = os.path.join(path, "ords_prods_merge_4_8.pkl")
ords_prods_merge.to_pickle(export_path)
export_path

OSError: [Errno 28] No space left on device

# Sanity Checks

In [24]:
int(ords_prods_merge["spending_flag"].value_counts(dropna=False).sum()), \
int(ords_prods_merge["frequency_flag"].value_counts(dropna=False).sum())

(36210749, 36210749)